databricks notebook3: create Customer dimension table

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

In [ ]:
delta_trip_path = "abfss://trip-data@<your-storage-account>.dfs.core.windows.net/silver/trip_transactions"

In [ ]:
# read data to dataframe
trip_df = spark.read.format('delta').load(delta_trip_path)

In [ ]:
# extract rider(customer) data 
customer_df = trip_df.select('customer_id','customer_name',
                             'payment_method','payment_status','total_fare').dropDuplicates(['customer_id']) 

In [ ]:
# create Customer dimension dataframe, and generate a new customerID surrogate key column
dim_customer = customer_df.withColumn('KeycustomerID',monotonically_increasing_id())

# reorder columns
dim_customer = dim_customer.select('KeycustomerID','customer_id','customer_name',
                             'payment_method','payment_status','total_fare')

In [ ]:
# write data to silver layer
dim_customer.write.format('delta').mode('overwrite').save("abfss://trip-data@<your-storage-account>.dfs.core.windows.net/silver/dim_customerd5" \
"")